In [ ]:
from fomo.pipelines.train import Learner
from fomo.pipelines.types.learner_args import LearnerArgs
from fomo.utils.data.datasets import DatasetInitializer, PROMPTS
import torch

NUM_OF_SAMPLES = 16

#TRAIN_SUBSAMPLES = ["base", "all"]
#TRAIN_SUBSAMPLES = ["base"]
TRAIN_SUBSAMPLES = ["base"]
MODEL_TYPES = ["clip_transformer_adapter_text", "clip_adapter"]


#DATASETS = ["cifar10", "caltech101", "oxford_pets", "oxford_flowers", "food101", "stanford_cars", "fgvc_aircraft", "dtd", "sun397"]
#DATASETS = ["caltech101", "oxford_pets", "oxford_flowers", "food101", "stanford_cars", "fgvc_aircraft", "dtd", "sun397"]
# DATASETS = ["imagenet"]
DATASETS = [
"cifar10",
# "caltech101",
# "oxford_pets",
# "oxford_flowers",
# "food101",
# "stanford_cars",
# "fgvc_aircraft",
# "sun397",
# "dtd",
# "eurosat",
# "ucf101",
]

for train_subsample in TRAIN_SUBSAMPLES:
    for model_type in MODEL_TYPES:
        for dataset_name in DATASETS:
            print(f"Training {model_type} on {dataset_name} on {train_subsample} subsample.")

            _dataset = DatasetInitializer.from_str(dataset_name).value(train=True)
            labels = _dataset.labels
            prompt_template = PROMPTS[dataset_name]
            num_of_classes = len(labels) // 2 if train_subsample == "base" else len(labels)
            #num_of_classes = len(labels) - (len(labels) // 5) if train_subsample == "base" else len(labels)
            train_eval_size = [NUM_OF_SAMPLES * num_of_classes, NUM_OF_SAMPLES * num_of_classes]

            learner_args = LearnerArgs(model_type=model_type, dataset=dataset_name)
            learner_args.device = "cuda"
            learner_args.epochs = 200
            learner_args.model_type = model_type
            learner_args.train_eval_size = train_eval_size
            learner_args.text_prompt_template = prompt_template
            learner_args.print_freq = 200
            learner_args.save_freq = 200
            learner_args.learning_rate = 5e-3
            learner_args.use_wandb = True
            learner_args.patience = 10
            learner_args.info = "dataset extensions"
            learner_args.batch_size = 256
            learner_args.momentum = 0.5
            learner_args.train_subsample = train_subsample
            learner_args.dataset = dataset_name
            learner_args.gaussian_noise_std = 0.3

            learner = Learner(learner_args)
            learner.run()